In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt

print(tf.__version__)
print(keras.__version__)

# Classification with Tensorflow and Keras

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [ ]:
print(X_train_full.shape)
print(X_train_full.dtype)
print(X_test.shape)
print(X_test.dtype)

In [ ]:
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

class_name = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", 
              "Sandal", "Shirt", "Sneaker", "Bag", "Ankled boot"]

In [ ]:
print(X_valid.shape)
print(X_valid.dtype)
print(X_train.shape)
print(X_train.dtype)

In [ ]:
model = keras.models.Sequential([
    # keras.layers.InputLayer(input_shape=[28, 28]),
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model.summary()

In [ ]:
model.layers

In [ ]:
hidden1 = model.layers[1]

print(hidden1.name)
w, b = hidden1.get_weights()
print(w.shape)
print(b.shape)

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0, 1]
plt.show()

In [ ]:
model.evaluate(X_test, y_test)

# Load and Pre-process Images

In [ ]:
import numpy as np
import os
import PIL
from PIL import Image
import tensorflow_datasets as tfds

In [ ]:
import pathlib

In [ ]:
# dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
# archive = tf.keras.utils.get_file(origin=dataset_url, extract=True)
# data_dir = pathlib.Path(archive).with_suffix('')

# print(archive)
# print(data_dir)

In [ ]:
data_path = pathlib.Path('data/train/').with_suffix('')
print(data_path)

In [ ]:
images_count = len(list(data_path.glob('*/*.jpg')))
print(images_count)

In [ ]:
dogs = list(data_path.glob('dog/*'))
Image.open(str(dogs[1]))

## Create a dataset

In [ ]:
batch_size = 20
img_height = 200
img_width = 200

In [ ]:
train_dataset = keras.utils.image_dataset_from_directory(
    data_path,
    validation_split=0.2,
    subset='training',
    seed=84,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

In [ ]:
val_dataset = keras.utils.image_dataset_from_directory(
    data_path,
    validation_split=0.2,
    subset='validation',
    seed=84,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

In [ ]:
class_names = train_dataset.class_names
print(class_names)

In [ ]:
# train_ds, test_ds = keras.utils.split_dataset(train_dataset, left_size=0.8)

## Visualize the data

In [ ]:
plt.figure(figsize=(10,10))
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
for img_batch, labels_batch in train_dataset:
    print(img_batch.shape)
    print(labels_batch.shape)
    break

## Standardize the data

In [ ]:
normalization_layer = keras.layers.Rescaling(1./255)
# To scale pixel values to [-1,1], use: tf.keras.layers.Rescaling(1./127.5, offset=-1)

## Configure the dataset for performance

- `Dataset.cache` keeps the images in memory after they're loaded off disk during the first epoch. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache.  
- `Dataset.prefetch` overlaps data preprocessing and model execution while training.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

# Train the Model

In [ ]:
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=[200, 200, 3]),
    keras.layers.Rescaling(1./255),
    keras.layers.Conv2D(
        filters=6,
        kernel_size=5,
        activation='relu'
    ), # 196 196, 6
    keras.layers.MaxPooling2D(), # 98, 98, 6
    keras.layers.Conv2D( 
        filters=18,
        kernel_size=3,
        activation='relu'
    ), # 96, 96, 18
    keras.layers.MaxPooling2D(), # 48, 48, 18
    keras.layers.Conv2D(
        filters=10,
        kernel_size=3,
        activation='relu'
    ), # 46, 46, 10
    keras.layers.MaxPooling2D(), # 23, 23, 10
    keras.layers.Flatten(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3
)

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0, 1]
plt.show()